# FIND USER'S POSTS

TODO: add refresh at every error , if the error persists after 3 refresh then quit

#### Import necessary libraries

In [18]:
# for scraping 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import xlsxwriter

# misc
import re as re # regex 
import time
import pandas as pd

# accesing env file 
import os 
from dotenv import load_dotenv # to access the secret keys we've hidden in a separate file 
load_dotenv() # grab values inside env file

True

#### Mercury Drugstore Setup

In [19]:
PATH = os.getenv("WEBDRIVER_PATH")

In [20]:
# initialize web driver that would control the web browser
ser = Service(PATH)
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)

#website we wanted to access 
driver.get("https://www.mercurydrug.com/store-locator.html")


#### Web Scraping Process

In [ ]:
# Creating lists
branch_name = []
branch_phone = []
branch_address = []
branch_store_hours = []

def mercury_drugs_scrape():
    group_iteration = 4
    group_frames = {
        "0" : ["luzframe","luzbranchframe","luzdetailframe"], 
        "1" : ["mmframe","mmbranchframe","mmdetailframe"], 
        "2" : ["visframe","visbranchframe","visdetailframe"], 
        "3" : ["minframe","minbranchframe","mindetailframe"],
    }
    group_frames_iteration = 3
    
    groups_size = range(1)

    for group in groups_size:

        # select group 
        driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//body/div[@class='container']/div/div/ul[@class='cf sf-loc-menu']/li[@data-tab='tab-{}']".format(group_iteration)))))

        # grab list of provinces 
        driver.switch_to.frame(group_frames["{}".format(group_frames_iteration)][0])
        iframe_province_page = driver.page_source
        soup = bs(iframe_province_page.encode("utf-8"), "html")
        soup.prettify()
        provinces = soup.findAll("li",{"class":None})

        provinceIteration = 1
        for province in provinces:
            # select province
            driver.find_element(By.CSS_SELECTOR,"li:nth-child({})".format(provinceIteration)).click()

            # grab list of cities 
            driver.switch_to.parent_frame() # go back to original frame 
            driver.switch_to.frame(group_frames["{}".format(group_frames_iteration)][1])
            iframe_cities_page = driver.page_source
            soup = bs(iframe_cities_page.encode("utf-8"), "html")
            soup.prettify()
            cities_box = soup.find("select",{"id":"sel1"})
            cities = cities_box.findAll("option",{"class":None})
            cities = cities[1:]

            cityIteration = 2
            for city in cities: 
                time.sleep(1)
                # select city 
                driver.find_element(By.CSS_SELECTOR,"option:nth-child({})".format(cityIteration)).click()

                # data scraping - here's where the fun starts 
                # switch to details frame, to grab details for each city 
                driver.switch_to.parent_frame() # go back to original frame 
                driver.switch_to.frame(group_frames["{}".format(group_frames_iteration)][2])
                iframe_details_page = driver.page_source
                soup = bs(iframe_details_page.encode("utf-8"), "html")
                soup.prettify()
                
                # grab the values 
                details_box = soup.find("div",{"class":"prov-details"})
                details_table = details_box.findAll("tr")
                branch_name_value = details_box.find("h4",{"class":None}).get_text()
                branch_address_value = details_table[0].select("td:nth-child(2)")[0].get_text()
                branch_phone_value = details_table[1].select("td:nth-child(2)")[0].get_text()
                branch_store_hours_value = details_table[2].select("td:nth-child(2)")[0].get_text()
                
                # append values 
                branch_name.append(branch_name_value)
                branch_phone.append(branch_phone_value)
                branch_address.append(branch_address_value)
                branch_store_hours.append(branch_store_hours_value)
                
                driver.switch_to.parent_frame() # go back to original frame 
                driver.switch_to.frame(group_frames["{}".format(group_frames_iteration)][1])

                # stop iterating once all cities are scraped
                option_size = len(city.findAll("option",{"class":None})[1:])

                if (cityIteration <= option_size + 1):
                    break

                cityIteration += 1
                continue

            # stop iterating once all province are scraped 
            # go back to province iframe 
            driver.switch_to.parent_frame() 
            driver.switch_to.frame(group_frames["{}".format(group_frames_iteration)][0])
            provinceIteration += 1
            continue

        # Storing scraped data into specified file format
        data = {
            "Name": branch_name,
            "Address": branch_address,
            "Phone": branch_phone,
            "Store Hours": branch_store_hours
        }

        # save to excel 
        df = pd.DataFrame(data)
        writer = pd.ExcelWriter("../data/mercury - mindanao.xlsx", engine='xlsxwriter')
        df.to_excel(writer, index =False)
        writer.save()
        
        if (group_iteration == groups_size):
            driver.quit()
            break
        
        driver.switch_to.parent_frame() 
        group_iteration += 1
        group_frames_iteration += 1
        continue         

#### Execute scraping

In [ ]:
mercury_drugs_scrape()

#### Storing scraped data into specified file format

In [ ]:
# data = {
#     "Name": branch_name,
#     "Address": branch_address,
#     "Phone": branch_phone,
#     "Store Hours": branch_store_hours
# }

# # save to excel 
# df = pd.DataFrame(data)
# writer = pd.ExcelWriter("../data/mercury.xlsx", engine='xlsxwriter')
# df.to_excel(writer, index =False)
# writer.save()